# Лабораторная работа №1 (Богомолов Михаил, гр.153502) « Случайные события и их имитация »
### В этой лабораторной работе вам необходимо разработать генераторы,
### которые имитируют происшествие различных случайных событий. В каждом
### задании вам нужно будет сгенерировать N = 10^6 значений, вычислить частоту
### выпадения каждого события и сравнить с теоретическими значениями.
### Реализация генераторов вынесена в файл custom_random.py

In [2]:
from custom_random import CustomRandom, simple_event_generator, complex_event_generator, complex_event_of_dependent_events_generator, full_group_event_generator
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

# Задание 1. Имитация простого случайного события
### На вход генератора подаёстся вероятность простого случайного события.
### В результате работы генератор должен вернуть с заданной вероятностью True,
### если событие произошло, и False, если событие не произошло.

In [3]:
prob_t1 = 0.197

generator_t1 = simple_event_generator(prob_t1)

N = 10**6
true_count_t1 = 0

for _ in range(N):
    if next(generator_t1):
        true_count_t1 += 1

true_experimental_freq_t1 = true_count_t1 / N
false_experimental_freq_t1 = 1 - true_experimental_freq_t1

true_theoretical_freq_t1 = prob_t1
false_theoretical_freq_t1 = 1 - true_theoretical_freq_t1

data_t1 = {'Событие': ['Произошло (True)', 'Не произошло (False)'],
    'Экспериментальная частота': [true_experimental_freq_t1, false_experimental_freq_t1],
    'Теоретическая частота': [true_theoretical_freq_t1, false_theoretical_freq_t1]}

df_t1 = pd.DataFrame(data_t1)
df_t1

,Событие,Экспериментальная частота,Теоретическая частота
0,Произошло (True),0.197317,0.197000
1,Не произошло (False),0.802683,0.803000


# Задание 2. Имитация сложного события
### На вход генератора подается список, содержащий вероятности k
### случайных независимых событий. В результате работы генератор должен
### вернуть список значений True/False, i-ый элемент которого соответствует
### выпадению/невыпадению i-го случайного события с заданной вероятностью.


In [5]:
probs_t2 = [0.116, 0.983, 0.011, 0.530, 0.678]

generator_t2 = complex_event_generator(probs_t2)

true_counts_t2 = [0] * len(probs_t2)

for _ in range(N):
    events = next(generator_t2)
    for index, event in enumerate(events):
        if event:
            true_counts_t2[index] += 1

true_experimental_freq_t2 = [count/N for count in true_counts_t2]
false_experimental_freq_t2 = [1 - freq for freq in true_experimental_freq_t2]


true_theoretical_freq_t2 = probs_t2
false_theoretical_freq_t2 = [1 - freq for freq in true_theoretical_freq_t2]

events_t2 = [f'Событие {i+1}' for i in range(len(probs_t2))]

data_true_t2 = {
    'Событие': events_t2,
    'Экспериментальная частота (True)': true_experimental_freq_t2,
    'Теоретическая частота (True)': true_theoretical_freq_t2
}
df_true_t2 = pd.DataFrame(data_true_t2)

data_false_t2 = {
    'Событие': events_t2,
    'Экспериментальная частота (False)': false_experimental_freq_t2,
    'Теоретическая частота (False)': false_theoretical_freq_t2
}
df_false_t2 = pd.DataFrame(data_false_t2)

df_combined_t2 = pd.concat([df_true_t2, df_false_t2], axis=1)

df_combined_t2

,Событие,Экспериментальная частота (True),Теоретическая частота (True),Событие,Экспериментальная частота (False),Теоретическая частота (False)
0,Событие 1,0.116150,0.116000,Событие 1,0.883850,0.884000
1,Событие 2,0.982963,0.983000,Событие 2,0.017037,0.017000
2,Событие 3,0.011280,0.011000,Событие 3,0.988720,0.989000
3,Событие 4,0.529716,0.530000,Событие 4,0.470284,0.470000
4,Событие 5,0.678435,0.678000,Событие 5,0.321565,0.322000


# Задание 3. Имитация сложного события, состоящего из зависимых событий
### На вход генератора подается вероятность Р(А) и условная вероятность
### Р(В|А). В результате работы генератор должен вернуть индикатор (число 0, 1, 2 или 3) 
### одного из четырех событий АB, Аnot(В), not(А)В, not(А)not(В) с соответствующими
### вероятностями Р(АВ), Р(Аnot(В)), Р(not(А)В), Р(not(А)not(В)). 
### Для теоретических расчетов указанных вероятностей воспользуйтесь формулой полной вероятности и
### теоремой Байеса. При выполнении используйте Р(not(А)) = 1 - Р(А) и Р(В|not(А)) = 1- Р(В|А).

In [7]:
p_a_t3 = 0.391
p_b_given_a_t3 = 0.599

generator_t3 = complex_event_of_dependent_events_generator(p_a_t3, p_b_given_a_t3)
true_counts_t3 = [0, 0, 0, 0]

true_theoretical_freq_t3 = [0, 0, 0, 0]
true_theoretical_freq_indexes_t3 = [-1, -1 , -1, -1]

for _ in range(N):
    outcome = next(generator_t3)
    true_counts_t3[outcome[0]] += 1
    if outcome[0] not in true_theoretical_freq_indexes_t3:
        true_theoretical_freq_indexes_t3[outcome[0]] = outcome[0]
        true_theoretical_freq_t3[outcome[0]] = outcome[1]
    
true_experimental_freq_t3 = [count/N for count in true_counts_t3]
false_experimental_freq_t3 = [1 - freq for freq in true_experimental_freq_t3]

false_theoretical_freq_t3 = [1 - freq for freq in true_theoretical_freq_t3]

events_t3 = ["АB", "Аnot(В)", "not(А)В", "not(А)not(В)"]


data_true_t3 = {
    'Событие': events_t3,
    'Экспериментальная частота (True)': true_experimental_freq_t3,
    'Теоретическая частота (True)': true_theoretical_freq_t3
}

df_true_t3 = pd.DataFrame(data_true_t3)

data_false_t3 = {
    'Событие': events_t3,
    'Экспериментальная частота (False)': false_experimental_freq_t3,
    'Теоретическая частота (False)': false_theoretical_freq_t3
}

df_false_t3 = pd.DataFrame(data_false_t3)

df_combined_t3 = pd.concat([df_true_t3, df_false_t3], axis=1)
df_combined_t3

,Событие,Экспериментальная частота (True),Теоретическая частота (True),Событие,Экспериментальная частота (False),Теоретическая частота (False)
0,АB,0.233872,0.234209,АB,0.766128,0.765791
1,Аnot(В),0.156853,0.156791,Аnot(В),0.843147,0.843209
2,not(А)В,0.244544,0.244209,not(А)В,0.755456,0.755791
3,not(А)not(В),0.364731,0.364791,not(А)not(В),0.635269,0.635209


# Задание 4. Имитация событий, образующих полную группу
### На вход генератора подается список, содержащий вероятности
### k случайных независимых событий, образующих полную группу. В результате
### своей работы генератор должен с заданными вероятностями вернуть
### индикатор (0, 1,...,k — 1) произошедшего на данном испытании события.

In [8]:
probs_t4 = [0.124, 0.035, 0.257, 0.506, 0.078]

generator_t4 = full_group_event_generator(probs_t4)

true_counts_t4 = [0] * len(probs_t4)

for _ in range(N):
    outcome = next(generator_t4)
    true_counts_t4[outcome] += 1
    
true_experimental_freq_t4 = [count/N for count in true_counts_t4]
false_experimental_freq_t4 = [1 - freq for freq in true_experimental_freq_t4]

true_theoretical_freq_t4 = probs_t4
false_theoretical_freq_t4 = [1 - freq for freq in true_theoretical_freq_t4]

events_t4 = [f'Событие {i+1}' for i in range(len(probs_t4))]

data_true_t4 = {
    'Событие': events_t4,
    'Экспериментальная частота (True)': true_experimental_freq_t4,
    'Теоретическая частота (True)': true_theoretical_freq_t4
}
df_true_t4 = pd.DataFrame(data_true_t4)

data_false_t4 = {
    'Событие': events_t4,
    'Экспериментальная частота (False)': false_experimental_freq_t4,
    'Теоретическая частота (False)': false_theoretical_freq_t4
}
df_false_t4 = pd.DataFrame(data_false_t4)

df_combined_t4 = pd.concat([df_true_t4, df_false_t4], axis=1)

df_combined_t4

,Событие,Экспериментальная частота (True),Теоретическая частота (True),Событие,Экспериментальная частота (False),Теоретическая частота (False)
0,Событие 1,0.124130,0.124000,Событие 1,0.875870,0.876000
1,Событие 2,0.034861,0.035000,Событие 2,0.965139,0.965000
2,Событие 3,0.257509,0.257000,Событие 3,0.742491,0.743000
3,Событие 4,0.505267,0.506000,Событие 4,0.494733,0.494000
4,Событие 5,0.078233,0.078000,Событие 5,0.921767,0.922000
